In [31]:
!git clone https://github.com/frgfm/torch-cam.git
!pip install -e torch-cam/.
!pip install grad-cam torchcam
!pip install torchcam

fatal: destination path 'torch-cam' already exists and is not an empty directory.
Obtaining file:///content/torch-cam
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for torchcam (pyproject.toml) ... done
  Created wheel for torchcam: filename=torchcam-0.4.1.dev0-0.editable-py3-none-any.whl size=16317 sha256=5a5c33b77c6377ea0ea2f8a436eb4fe2fae247870f929f9b9aff4d2189920e36
  Stored in directory: /tmp/pip-ephem-wheel-cache-r34rko46/wheels/01/be/d9/bcd58c55f0941c76ea49be4bc6b7a29cbf46c236c38c1b3b98
Successfully built torchcam
  Attempting uninstall: torchcam
    Found existing installation: torchcam 0.4.1.dev0
    Uninstalling torchcam-0.4.1.dev0:
      Successfully uninstalled torchcam-0.4.1.dev0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/audrey_dataset_plants'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

Import

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Set device

In [ ]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  return torch.device(dev)

In [ ]:
# print('device', device)

Load pretrained model

In [6]:
from torchvision import models

In [ ]:
# model = models.resnet18(pretrained=True)

In [ ]:
# model = model.eval()
# model = model.to(device)

Get data to normalize

In [7]:
from torchvision import transforms
from torchvision import datasets

In [8]:
# Get the data from your train dataset for getting the mean and std
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.ToTensor()])

training_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=False)

In [9]:
def get_mean_and_std(loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    count = 0

    # Loop over the images in the DataLoader
    for data in train_loader:
        # Apply the transformation to the image
        img = data[0]


        # Compute the mean and standard deviation of the pixel values in the image
        mean += torch.mean(img,dim=(0,2,3))
        std += torch.std(img,dim=(0,2,3))
        count += 1

    # Compute the mean and standard deviation across all images
    mean /= count
    std /= count

    print('Mean:', mean)
    print('Std:', std)
    return mean,std

In [ ]:
# after run this for one time, just record this data and comment out this part
get_mean_and_std(train_loader)

Preprocess data

In [10]:
from torchvision import transforms
from torchvision import datasets

In [11]:
# set up the transforms for the datasets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5188, 0.4596, 0.3459],
                                                            [0.2735, 0.2508, 0.2684])])
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5188, 0.4596, 0.3459],
                                                           [0.2735, 0.2508, 0.2684])])

training_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

Load the json file

In [12]:
import json

with open('/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/audrey_dataset_plants/flower_types.json', 'r') as f:
  classes_name = json.load(f)

print(len(classes_name))
print(classes_name)

3
{'0': 'dandelion', '1': 'poison ivy', '2': 'tulip'}


Training functions

In [13]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
  device = set_device()
  best_acc = 0

  for epoch in range(n_epochs):
    print("Epoch number %d" % (epoch + 1))
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    total = 0

    for data in train_loader:
      images, labels = data
      images, labels = images.to(device), labels.to(device)
      total += labels.size(0)

      optimizer.zero_grad()

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      loss = criterion(outputs, labels)

      loss.backward()

      optimizer.step()

      running_loss += loss.item()
      running_correct += (labels==predicted).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100.00 * running_correct / total

    print("     = Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f"
          % (running_correct, total, epoch_acc, epoch_loss))

    test_dataset_acc = evaluate_model_on_test_set(model, test_loader)

    if (test_dataset_acc > best_acc):
      best_acc = test_dataset_acc
      save_checkpoint(model, epoch, best_acc)

  print("Finished")
  return model

In [14]:
def evaluate_model_on_test_set(model, test_loader):
  model.eval()
  predicted_correctly_on_epoch = 0
  total = 0
  device = set_device()

  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images, labels = images.to(device), labels.to(device)
      total += labels.size(0)

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      predicted_correctly_on_epoch += (predicted==labels).sum().item()

  epoch_acc = 100.00 * predicted_correctly_on_epoch / total
  print("     = Test dataset. Got %d out of %d images correctly (%.3f%%)"
        % (predicted_correctly_on_epoch, total, epoch_acc))

  return epoch_acc

In [15]:
def save_checkpoint(model, epoch, best_acc):
  state = {
      'epoch' : epoch + 1,
      'model' : model.state_dict(),
      'best accuracy' : best_acc,
  }
  torch.save(state, 'model_best_checkpoint.pth.tar')

Set up training

In [16]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(pretrained=False)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 3
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet_18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [17]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, n_epochs=70)

Epoch number 1
     = Training dataset. Got 24 out of 87 images correctly (27.586%). Epoch loss: 1.206
     = Test dataset. Got 12 out of 37 images correctly (32.432%)
Epoch number 2
     = Training dataset. Got 46 out of 87 images correctly (52.874%). Epoch loss: 1.041
     = Test dataset. Got 15 out of 37 images correctly (40.541%)
Epoch number 3
     = Training dataset. Got 47 out of 87 images correctly (54.023%). Epoch loss: 0.918
     = Test dataset. Got 16 out of 37 images correctly (43.243%)
Epoch number 4
     = Training dataset. Got 72 out of 87 images correctly (82.759%). Epoch loss: 0.644
     = Test dataset. Got 12 out of 37 images correctly (32.432%)
Epoch number 5
     = Training dataset. Got 67 out of 87 images correctly (77.011%). Epoch loss: 0.537
     = Test dataset. Got 17 out of 37 images correctly (45.946%)
Epoch number 6
     = Training dataset. Got 69 out of 87 images correctly (79.310%). Epoch loss: 0.641
     = Test dataset. Got 18 out of 37 images correctly (4

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Load checkpoint

In [18]:
checkpoint = torch.load('model_best_checkpoint.pth.tar')

In [19]:
print(checkpoint['epoch'])
print(checkpoint['best accuracy'])

60
94.5945945945946


Save model

In [20]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 3
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
resnet18_model.load_state_dict(checkpoint['model'])

torch.save(resnet18_model, 'best_model.pth')

Testing for one picture

In [21]:
classes = [
    "dandelion",
    "poison ivy",
    "tulips"
]

In [22]:
model = torch.load('best_model.pth')

In [23]:
image_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize([0.5188, 0.4596, 0.3459],
                                                           [0.2735, 0.2508, 0.2684])])

In [24]:
import PIL.Image as Image

def classify(model, image_transforms, image_path, classes):
  model = model.eval()
  image = Image.open(image_path)
  image = image_transforms(image).float()
  image = image.unsqueeze(0)

  output = model(image)
  _, predicted = torch.max(output.data, 1)

  print(classes[predicted.item()])

In [25]:
classify (model, image_transforms, "/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/audrey_dataset_plants/vd1t.webp", classes)

tulips


CAM (Class Activated Map)


In [26]:
# from torchvision.models import resnet18
# model = resnet18(pretrained=True).eval().to(device)
model = torch.load('best_model.pth')
model = model.eval().to(device)

In [34]:
from torchcam.methods import SmoothGradCAMpp

cam_extractor = SmoothGradCAMpp(model)

ModuleNotFoundError: No module named 'torchcam'

In [35]:
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.5188, 0.4596, 0.3459],
                                         std=[0.2735, 0.2508, 0.2684])
                                    ])




In [36]:
img_path = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/audrey_dataset_plants/vd1t.webp'

In [37]:
img_pil = Image.open(img_path)
input_tensor = test_transform(img_pil).unsqueeze(0).to(device)

In [38]:
pred_logits = model(input_tensor)
pred_id = torch.topk(pred_logits, 1)[1].detach().cpu().numpy().squeeze().item()

In [39]:
pred_id

0

In [40]:
activation_map = cam_extractor(pred_id, pred_logits)
activation_map = activation_map[0][0].detach().cpu().numpy()

NameError: name 'cam_extractor' is not defined

In [ ]:
activation_map.shape

In [ ]:
plt.imshow(activation_map)
plt.show()

In [ ]:
from torchcam.utils import overlay_mask

result = overlay_mask(img_pil, Image.fromarray(activation_map), alpha=0.6)

In [ ]:
result

Grad cam

In [ ]:
from pytorch_grad_cam import GradCAM
# HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, EigenGradCAM, LayerCAM, FullGrad, GradCAMElementWise
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, deprocess_image, preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

model = torch.load('best_model.pth')
model = model.eval().to(device)

In [ ]:
from torchvision import transforms

test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.5188, 0.4596, 0.3459],
                                         std=[0.2735, 0.2508, 0.2684])
                                    ])

In [ ]:
img_path = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/audrey_dataset_plants/vd1t.webp'

img_pil = Image.open(img_path)
input_tensor = test_transform(img_pil).unsqueeze(0).to(device)
input_tensor.shape

In [ ]:
model.layer4[-1]

In [ ]:
target_layers = [model.layer4[-1]]
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

In [ ]:
with GradCAM(model=model, target_layers=target_layers, use_cuda=True) as cam:
    cam_map = cam(input_tensor=input_tensor, targets=None, aug_smooth=True, eigen_smooth=True)[0]

In [ ]:
cam_map.shape
plt.imshow(cam_map)
plt.show()

In [ ]:
import torchcam
from torchcam.utils import overlay_mask

result = overlay_mask(img_pil, Image.fromarray(cam_map), alpha=0.5)

In [ ]:
result

Live cam Grad cam

In [ ]:
# if not using openmim, it will stuck at install mmcv-full
!pip install openmim
!mim install mmcv-full

In [ ]:
import os
import time
import shutil
import tempfile
from tqdm import tqdm

import cv2
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import gc

from PIL import Image, ImageDraw, ImageFont

import torch
import torch.nn.functional as F
from torchvision import models

import mmcv


In [ ]:
# run everything in the backend
import matplotlib
matplotlib.use('Agg')

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.5188, 0.4596, 0.3459],
                                         std=[0.2735, 0.2508, 0.2684])
                                    ])

In [ ]:
dataset_dir = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/flower'
train_path = os.path.join(dataset_dir, 'train')
train_dataset = datasets.ImageFolder(train_path, train_transform)
class_names = train_dataset.classes
n_class = len(class_names)

In [ ]:
class_names

In [ ]:
# for make the class_name to .npy file later
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [ ]:
np.save('idx_to_labels.npy', idx_to_labels)
np.save('labels_to_idx.npy', train_dataset.class_to_idx)

In [ ]:
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()

In [ ]:
idx_to_labels

In [ ]:
model = torch.load('best_model.pth')
model = model.eval().to(device)

In [ ]:
def pred_single_frame(img, n=5):

    img_bgr = img
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR to RGB
    img_pil = Image.fromarray(img_rgb) # array to pil
    input_img = train_transform(img_pil).unsqueeze(0).to(device) # preprocess
    pred_logits = model(input_img) # get all different types' logit score
    pred_softmax = F.softmax(pred_logits, dim=1) # do softmax on logit scores

    top_n = torch.topk(pred_softmax, n) # get n number of results which are most possible
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # get the type index
    confs = top_n[0].cpu().detach().numpy().squeeze() # get the probability

    # write on the graph
    for i in range(n):
        class_name = idx_to_labels[pred_ids[i]] # get the name of that type
        confidence = confs[i] * 100 # get the probability in %
        text = '{:<15} {:>.4f}'.format(class_name, confidence)

        img_bgr = cv2.putText(img_bgr, text, (25, 50 + 40 * i), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 255), 3)

    return img_bgr, pred_softmax

In [ ]:
input_video = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/Resources/testing video/rose.mp4'
output_path = '/content/drive/MyDrive/CSE-16 SIP [ Summer 2023 ]/Resources/testing video/rose_result.mp4'

In [ ]:
# create a temp folder for store images
temp_out_dir = time.strftime('%Y%m%d%H%M%S')
os.mkdir(temp_out_dir)
print('Create temp folder to store'.format(temp_out_dir))

In [ ]:
imgs = mmcv.VideoReader(input_video)

prog_bar = mmcv.ProgressBar(len(imgs))

for frame_id, img in enumerate(imgs):

    img, pred_softmax = pred_single_frame(img, n=2)

    cv2.imwrite(f'{temp_out_dir}/{frame_id:06d}.jpg', img)

    prog_bar.update()

mmcv.frames2video(temp_out_dir, output_path, fps=imgs.fps, fourcc='mp4v')

shutil.rmtree(temp_out_dir)
print('Delete temp folder', temp_out_dir)
print('Finish video', output_path)